In [11]:
#Basic Libraries
import os
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#For image operation
import cv2
from PIL import Image

#For Pytorch
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.transforms import transforms
from torch.utils.data import Dataset, random_split, DataLoader, TensorDataset

#For SK Learn
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, RocCurveDisplay, accuracy_score, make_scorer, roc_auc_score
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.neural_network import MLPClassifier

#For Keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

from tqdm import tqdm

In [12]:
workingDir = 'E:\\Document\\M-Tech\\M-Tech Class\\2nd Trimester\\Assignment\\AI Lab\\Assignment 7\\Datasets\\ch22m503\\'
dataset_folderName = ['parasite', 'uninfected']
img_size = 150
seed = 42
dataset = []
label = []
epochs = 1000
round_precesion= 2

In [13]:
import torch
import torch.nn as nn

class MalariaCNN(nn.Module):
    def __init__(self):
        super(MalariaCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3)
        self.fc1 = nn.Linear(in_features=32*26*26, out_features=128)
        self.fc2 = nn.Linear(in_features=128, out_features=2)
    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 32*26*26)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = MalariaCNN()

In [14]:
model

MalariaCNN(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=21632, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=2, bias=True)
)

In [17]:
import os
from PIL import Image
from torch.utils.data import Dataset
from torch.utils.data import DataLoader 

class MalariaDataset(Dataset):
    def __init__(self, data_directory):
        self.data_directory = data_directory
        self.image_path = []
        self.labels = []
        for loop_label in os.listdir(data_directory):
            label_directory = data_directory + loop_label + '\\'#os.path.join(data_directory, loop_label)
            for img_name in os.listdir(label_directory):
                img_path = os.path.join(label_directory, img_name)
                self.image_path.append(img_path)
                self.labels.append(loop_label)

    def __len__(self):
        return len(self.image_path)

    def __getitem__(self, idx):
        img_path = self.image_path[idx]
        label = self.labels[idx]  
        transformer = transforms.Compose([
            transforms.Resize((64, 64)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
        ])
        with Image.open(img_path) as img:
            rgb_img = transformer(img.convert('RGB'))
        label = int(label == 'parasite')
        return rgb_img, label


    def split_data(self, size = 0.2):
        test_size = round(size * len(self.image_path))
        train_size = len(self.image_path) - test_size
        validation_size = round(size * train_size)
        train_size = train_size - validation_size
        return random_split(self, [train_size, validation_size,test_size])

# Create dataset and dataloaders
train_dir = 'E:\\Document\\M-Tech\\M-Tech Class\\2nd Trimester\\Assignment\\AI Lab\\Assignment 7\\Datasets\\ch22m503\\'
# test_dir = 'path/to/testing/folder'
train_dataset = MalariaDataset(train_dir)
# test_dataset = MalariaDataset(test_dir)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
# test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [18]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from tqdm import tqdm

# Define the CNN model
class MalariaCNN(nn.Module):
    def __init__(self):
        super(MalariaCNN, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3)
        self.fc1 = nn.Linear(in_features=32*14*14, out_features=128)
        self.fc2 = nn.Linear(in_features=128, out_features=2)
        
    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 32*14*14)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x


train_dataset = MalariaDataset(train_dir) 
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Initialize the CNN model and optimizer
model = MalariaCNN()
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# Train the model
num_epochs = 2
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
for epoch in range(num_epochs):
    running_loss = 0.0
    running_acc = 0.0
    for i, (inputs, labels) in enumerate(tqdm(train_loader)):
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        running_acc += torch.sum(preds == labels.data)
    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = running_acc.double() / len(train_dataset)
    print('Epoch [{}/{}], Loss: {:.4f}, Accuracy: {:.4f}'.format(epoch+1, num_epochs, epoch_loss, epoch_acc))


100%|██████████| 8/8 [00:01<00:00,  6.56it/s]


Epoch [1/2], Loss: 0.6948, Accuracy: 0.5305


100%|██████████| 8/8 [00:01<00:00,  6.65it/s]

Epoch [2/2], Loss: 0.6620, Accuracy: 0.5935
